In [1]:
import logging

from rlduels.src.DataHandling.trajectory_pair import Transition, Trajectory, TrajectoryPair, NDArray

from rlduels.src.DataHandling.database_manager import MongoDBManager

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

In [2]:
import gymnasium as gym

def run_cartpole():
    # Create the CartPole environment
    env = gym.make('CartPole-v1')

    # Define the number of episodes you want to run
    num_episodes = 5

    trajectories = []

    for episode in range(num_episodes):
        # Reset the environment for a new episode
        observation, info = env.reset()
        print("Observation:", observation)
        total_reward = 0
        done = False

        transitions = []

        while not done:
            # Render the environment to visualize the cart and pole
            env.render()

            # Here we select a random action (either 0 or 1; left or right)
            action = env.action_space.sample()

            # Take the action in the environment
            next_observation, reward, terminated, truncated, info = env.step(action)

            # Record the transition
            transitions.append(Transition(
                state=NDArray(array=observation),
                action=NDArray(array=action),
                reward=reward,
                terminated=terminated,
                truncated=truncated,
                next_state=NDArray(array=next_observation)
            ))

            observation = next_observation
            total_reward += reward
            done = terminated or truncated

        # Log the trajectory
        trajectories.append(Trajectory(
            information={},
            transitions=transitions
        ))

        if done:
            print(f"Episode {episode + 1}: Total reward = {total_reward}")

    # Close the environment
    env.close()

    # Return trajectories for further analysis if necessary
    return trajectories

In [3]:
trajs = run_cartpole()

x = TrajectoryPair(trajectory1=trajs[0], trajectory2=trajs[1])

Observation: [-0.00181753  0.02878489 -0.01235337  0.01637262]
Episode 1: Total reward = 24.0
Observation: [-0.04950432 -0.04788609 -0.01262357 -0.04005535]
Episode 2: Total reward = 27.0
Observation: [-0.02341092 -0.02141287  0.02497706  0.04089235]
Episode 3: Total reward = 13.0
Observation: [ 0.02685427 -0.04012443 -0.02983151 -0.01989495]
Episode 4: Total reward = 17.0
Observation: [-0.04600982 -0.03824271 -0.04757196  0.00647755]
Episode 5: Total reward = 28.0


/home/student/.miniforge3/envs/rlduels/lib/python3.8/site-packages/gymnasium/envs/classic_control/cartpole.py:215: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(


In [4]:
db = MongoDBManager()

2024-05-05 23:12:18 - ERROR - An error occurred while attempting to start MongoDB: name 'time' is not defined
Traceback (most recent call last):
  File "/home/student/Code/RLDuels/rlduels/src/DataHandling/database_manager.py", line 67, in start_db
    time.sleep(0.5)
NameError: name 'time' is not defined


In [5]:
print(db.add_entry(x))
print(db.find_entry(x))
print(db.delete_entry(x))
print(db.find_entry(x))

2024-05-05 23:12:18 - DEBUG - {"message": "Server selection started", "selector": "<function writable_server_selector at 0x75b2c12f4ee0>", "operation": "insert", "topologyDescription": "<TopologyDescription id: 6637f632329c345619fc9c0a, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>", "clientId": {"$oid": "6637f632329c345619fc9c0a"}}
2024-05-05 23:12:18 - INFO - {"message": "Waiting for suitable server to become available", "selector": "<function writable_server_selector at 0x75b2c12f4ee0>", "operation": "insert", "topologyDescription": "<TopologyDescription id: 6637f632329c345619fc9c0a, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 111] Connection refused (configured time

('Added a50ce798-813e-4e88-87f1-a30c33ce1e6b to the database', None)
id=UUID('2d8fe616-3a7a-4207-ad1d-63a8de80595c') trajectory1=Trajectory(information={}, transitions=[Transition(state=NDArray(array=array([-0.00181753,  0.02878489, -0.01235337,  0.01637262])), action=NDArray(array=array(0)), reward=1.0, terminated=False, truncated=False, next_state=NDArray(array=array([-0.00124183, -0.16615774, -0.01202592,  0.30513245]))), Transition(state=NDArray(array=array([-0.00124183, -0.16615774, -0.01202592,  0.30513245])), action=NDArray(array=array(1)), reward=1.0, terminated=False, truncated=False, next_state=NDArray(array=array([-0.00456499,  0.02913351, -0.00592327,  0.00868124]))), Transition(state=NDArray(array=array([-0.00456499,  0.02913351, -0.00592327,  0.00868124])), action=NDArray(array=array(1)), reward=1.0, terminated=False, truncated=False, next_state=NDArray(array=array([-0.00398232,  0.2243399 , -0.00574964, -0.28586465]))), Transition(state=NDArray(array=array([-0.00398232, 

In [6]:
db.close_db()

AttributeError: 'NoneType' object has no attribute 'terminate'

In [ ]:
import json

serialized_data = x.json()

data_to_store = json.loads(serialized_data)

data_to_store['_id'] = data_to_store.pop('id')

print(data_to_store)

{'trajectory1': {'information': {}, 'transitions': [{'state': {'array': [0.04219505190849304, 0.006960288621485233, -0.011760519817471504, -0.04491124302148819]}, 'action': {'array': 0}, 'reward': 1.0, 'terminated': False, 'truncated': False, 'next_state': {'array': [0.04233425483107567, -0.18799106776714325, -0.012658745050430298, 0.24403804540634155]}}, {'state': {'array': [0.04233425483107567, -0.18799106776714325, -0.012658745050430298, 0.24403804540634155]}, 'action': {'array': 1}, 'reward': 1.0, 'terminated': False, 'truncated': False, 'next_state': {'array': [0.03857443481683731, 0.007309386041015387, -0.007777984254062176, -0.05261072516441345]}}, {'state': {'array': [0.03857443481683731, 0.007309386041015387, -0.007777984254062176, -0.05261072516441345]}, 'action': {'array': 1}, 'reward': 1.0, 'terminated': False, 'truncated': False, 'next_state': {'array': [0.03872062265872955, 0.2025419920682907, -0.008830199018120766, -0.34773749113082886]}}, {'state': {'array': [0.03872062